In [1]:
import scipy.signal
import sys
import torch
import torch.nn as nn
import sys
sys.path.append("/home/xhnfly/Cosmic_rays_X/XRL/")
import numpy as np

In [2]:
from typing import Dict, List, Optional, Tuple
import gym
from PIL import Image

from pyvirtualdisplay import Display
Display().start()
from datetime import datetime

In [3]:
import math
import random
from copy import deepcopy
import torch
from torch.optim import Adam
from torch.optim import RMSprop
import XRL_main
import gym
import time
from collections import namedtuple, deque
import neptune.new as neptune

# from tqdm import tqdm

In [4]:
nep_log = neptune.init(
    project="xhnfirst/DDPG-mian-test1",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NTg5MDI2OS01MTVmLTQ2YjUtODA1Yy02ZWQyNDgxZDcwN2UifQ==",
)

https://app.neptune.ai/xhnfirst/DDPG-mian-test1/e/DDPGMIAN-109
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [5]:
import math
import sys
from time import strftime
from time import gmtime

class view_bar:
    def __init__(self, message, num, total, value_name, value_data, start_time_rec, r_true, reward_ctrl, reward_dist, total_main):
        self.message = message
        self.num = num
        self.total = total
        self.message2 = "True time of r = "
        self.r_true = r_true
        self.message3 = "time cost = "
        rate = self.num  / self.total
        rate_num = int(rate * 40)
        rate_nums = math.ceil(rate * 100)
        time_rec = datetime.now()
        execution_time = time_rec - start_time_rec
        self.message4 = "{reward_ctrl: "
        self.reward_ctrl = reward_ctrl
        self.message5 = "reward_dist: "
        self.reward_dist = reward_dist
        self.message6 = "Total Reward Main: "
        self.total_main = total_main
        self.message8 = " }"



        r = '\r%s:[%s%s]%d%%\t%d/%d\t%s%f\t%s%s\t%s%s\t\t\t\t%s%.4f\t%s%.4f\t%s%.4f\t%s' % (self.message, "=" * rate_num,
                                        " " * (40 - rate_num), rate_nums, self.num , self.total, value_name, value_data, 
                                        self.message3, execution_time, self.message2, str(self.r_true), str(self.message4), self.reward_ctrl,
                                        str(self.message5), self.reward_dist, str(self.message6), self.total_main, 
                                         str(self.message8)
                                        )


        sys.stdout.write(r)
        sys.stdout.flush()


# a =10
# for i in range(60000):
#     view_bar("epoch ", i+1, 60000, 'a = ', float(a))
#     a += 0.01


In [6]:
env = gym.make('Reacher-v2')
test_env = gym.make('Reacher-v2')
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device = ', device)

device =  cuda


In [7]:
def mlp(sizes, activation, output_activation=nn.Identity):
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    return nn.Sequential(*layers)


class MLPActor(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation, act_limit):
        super().__init__()
        pi_sizes = [obs_dim] + list(hidden_sizes) + [act_dim]
        self.pi = mlp(pi_sizes, activation, nn.Tanh)
        self.act_limit = act_limit

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        return self.act_limit * self.pi(obs)

class MLPQFunction(nn.Module):

    def __init__(self, obs_dim, act_dim, hidden_sizes, activation):
        super().__init__()
        self.q = mlp([obs_dim + act_dim] + list(hidden_sizes) + [1], activation)

    def forward(self, obs, act):
        q = self.q(torch.cat([obs, act], dim=-1))
        return torch.squeeze(q, -1) # Critical to ensure q has right shape.

class MLPActorCritic(nn.Module):

    def __init__(self, observation_space, action_space, hidden_sizes=(256,256),
                 activation=nn.ReLU, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        super().__init__()

        obs_dim = observation_space.shape[0]
        act_dim = action_space.shape[0]
        act_limit = action_space.high[0]

        # build policy and value functions
        self.pi = MLPActor(obs_dim, act_dim, hidden_sizes, activation, act_limit).to(device)
        self.q = MLPQFunction(obs_dim, act_dim, hidden_sizes, activation).to(device)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs)

In [8]:
Transition = namedtuple('Transition',
                        ('obs', 'act', 'rew', 'next_obs', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [9]:

params = {
    "batch_size": 128,
    "dropout": 0.2,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "hid": 64,
    "l": 3,
    "seed": 0,
    "steps_per_epoch": 3000,
    "epochs": 100,
    "replay_size": int(1e6),
    "gamma": 0.99,
    "polyak": 0.995,
    "pi_lr": 1e-4,
    "q_lr": 1e-4,
    "batch_size": 500,
    "start_steps": 10000, 
    "update_after": 1000,
    "update_every": 50,
    "act_noise": 0.01,
    "num_test_episodes": 10,
    "max_ep_len": 1000
}


ac_kwargs=dict(hidden_sizes=[params["hid"]]*params["l"])



In [10]:
nep_log["parameters"] = params



torch.manual_seed(params["seed"])
np.random.seed(params["seed"])
obs_dim = env.observation_space.shape[0]

print('obs_dim = ', obs_dim)
act_dim = env.action_space.shape[0]
print('act_dim = ', act_dim)
# Action limit for clamping: critically, assumes all dimensions share the same bound!
act_limit = env.action_space.high[0]
print('act_limit = ', act_limit)
# Create actor-critic module and target networks
ac = MLPActorCritic(env.observation_space, env.action_space, **ac_kwargs)
ac_targ = deepcopy(ac)

# Freeze target networks with respect to optimizers (only update via polyak averaging)
for p in ac_targ.parameters():
    p.requires_grad = False

memory = ReplayMemory(params["replay_size"])

obs_dim =  11
act_dim =  2
act_limit =  1.0


In [11]:
# Set up function for computing DDPG Q-loss
def compute_loss_q(data):

    o = torch.cat(data.obs).float()
    a = torch.cat(data.act).float()
    r = torch.cat(data.rew).float()
    o2 =torch.cat(data.next_obs).float()
    d = torch.cat(data.done).float()

    q = ac.q(o,a)


    # Bellman backup for Q function
    with torch.no_grad():
        q_pi_targ = ac_targ.q(o2, ac_targ.pi(o2))
        backup = r + params["gamma"] * (1 - d) * q_pi_targ

    # MSE loss against Bellman backup
    loss_q = ((q - backup)**2).mean()

    return loss_q

# Set up function for computing DDPG pi loss
def compute_loss_pi(data):

    o = torch.cat(data.obs).float()

    q_pi = ac.q(o, ac.pi(o))

    return -q_pi.mean()


In [12]:
pi_optimizer = RMSprop(ac.pi.parameters(), lr=params["pi_lr"])
q_optimizer = RMSprop(ac.q.parameters(), lr=params["q_lr"])

def update(data):
    # First run one gradient descent step for Q.


    q_optimizer.zero_grad()
    loss_q = compute_loss_q(data)

    loss_q.backward()

    q_optimizer.step()


    # Freeze Q-network so you don't waste computational effort 
    # computing gradients for it during the policy learning step.
    for p in ac.q.parameters():
        p.requires_grad = False

    # Next run one gradient descent step for pi.
    pi_optimizer.zero_grad()
    loss_pi = compute_loss_pi(data)
    loss_pi.backward()
    pi_optimizer.step()

    # Unfreeze Q-network so you can optimize it at next DDPG step.
    for p in ac.q.parameters():
        p.requires_grad = True

    nep_log


    # Finally, update target networks by polyak averaging.
    with torch.no_grad():
        for p, p_targ in zip(ac.parameters(), ac_targ.parameters()):
            # NB: We use an in-place operations "mul_", "add_" to update target
            # params, as opposed to "mul" and "add", which would make new tensors.
            p_targ.data.mul_(params["polyak"])
            p_targ.data.add_((1 - params["polyak"]) * p.data)


In [13]:



def get_action(o, noise_scale):
    a = ac.act(torch.tensor(o, dtype=torch.float32, device=device))
    a += noise_scale * torch.randn(act_dim).to(device)
    return torch.clip(a, -act_limit, act_limit)

def test_agent(epoch):
    screen = test_env.render(mode='rgb_array')
    im = Image.fromarray(screen)
    images = [im]
    for j in range(params["num_test_episodes"]):
        o, d, test_ep_ret, test_ep_len = test_env.reset(), False, 0, 0
        if j == 9:
            screen = test_env.render(mode='rgb_array')
            time.sleep(0.02)
        while not(d or (test_ep_len == params["max_ep_len"])):
            # Take deterministic actions at test time (noise_scale=0)
            a_cpu = get_action(o, 0).cpu().data.numpy()
            # a = get_action(o, 0)
            if j == 9:
                screen = test_env.render(mode='rgb_array')
                images.append(Image.fromarray(screen))
                time.sleep(0.02)
            o, r, d, _ = test_env.step(a_cpu)
            # if np.abs(reward_dict["reward_dist"]) < 0.1 and np.abs(reward_dict["reward_ctrl"]) < 1.5:
            #     r += 2
            #     if np.abs(reward_dict["reward_dist"]) < 0.05 and np.abs(reward_dict["reward_ctrl"]) < 1:
            #         r += 4
            #         if np.abs(reward_dict["reward_dist"]) < 0.03 and np.abs(reward_dict["reward_ctrl"]) < 0.7:
            #             r += 8
            #             if np.abs(reward_dict["reward_dist"]) < 0.02 and np.abs(reward_dict["reward_ctrl"]) < 0.4:
            #                 r += 16
            #                 if np.abs(reward_dict["reward_dist"]) < 0.01 and np.abs(reward_dict["reward_ctrl"]) < 0.2:
            #                     r += 32
            #                     if np.abs(reward_dict["reward_dist"]) < 0.005 and np.abs(reward_dict["reward_ctrl"]) < 0.05:
            #                         r += 64
            #                         if np.abs(reward_dict["reward_dist"]) < 0.001 and np.abs(reward_dict["reward_ctrl"]) < 0.01:
            #                             r += 128
            #                             if np.abs(reward_dict["reward_dist"]) < 0.0001 and np.abs(reward_dict["reward_ctrl"]) < 0.001:
            #                                 r += 256
            # else:
            #     r += -0.1
            test_ep_ret += r
            test_ep_len += 1
        nep_log["test/reward"].log(test_ep_ret/test_ep_len)
    
    now = datetime.now()

    current_time = str(now.isoformat())
    image_file = 'video/DDPG_reacher_%s_ep_%d.gif' % (current_time, epoch)
    images[0].save(
                    image_file, save_all=True, 
                    append_images=images[1:], 
                    loop=0, duration=1)



In [14]:
# Prepare for interaction with environment
total_steps = params["steps_per_epoch"] * params["epochs"]
start_time = time.time()
o, ep_ret, ep_len = env.reset(), 0, 0
o = torch.tensor([o], device=device)


start_time_rec = datetime.now()
r_true = 0
total_main = 0
ep_ret_main = 0
reward_dict={}

/tmp/ipykernel_11405/296350107.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/torch/csrc/utils/tensor_new.cpp:201.)
  o = torch.tensor([o], device=device)


In [15]:
# Main loop: collect experience in env and update/log each epoch


for t in range(total_steps):
    
    # Until start_steps have elapsed, randomly sample actions
    # from a uniform distribution for better exploration. Afterwards, 
    # use the learned policy (with some noise, via act_noise). 
    if t > params["start_steps"]:
        a = get_action(o, params["act_noise"])
        # a_cpu = a
        a_cpu = a.cpu().data.numpy()
        
    else:
        a = env.action_space.sample()
        a_cpu = a
        a = torch.tensor([a], device=device)
    

    # Step the env
    o2, r, d, reward_dict= env.step(a_cpu)
    print('o2 = ', o2)

    nep_log["testreward/dist"].log(reward_dict["reward_dist"])
    nep_log["testreward/ctrl"].log(reward_dict["reward_ctrl"])
        
    # r = np.log(np.abs(reward_dict["reward_ctrl"]))*(-0.01) + np.log(np.abs(reward_dict["reward_dist"]))*(-0.1)

    # if np.abs(reward_dict["reward_dist"]) < 0.1 and np.abs(reward_dict["reward_ctrl"]) < 1.5:
    #     r += 2
    #     if np.abs(reward_dict["reward_dist"]) < 0.05 and np.abs(reward_dict["reward_ctrl"]) < 1:
    #         r += 4
    #         if np.abs(reward_dict["reward_dist"]) < 0.03 and np.abs(reward_dict["reward_ctrl"]) < 0.7:
    #             r += 8
    #             if np.abs(reward_dict["reward_dist"]) < 0.02 and np.abs(reward_dict["reward_ctrl"]) < 0.4:
    #                 r += 16
    #                 if np.abs(reward_dict["reward_dist"]) < 0.01 and np.abs(reward_dict["reward_ctrl"]) < 0.2:
    #                     r += 32
    #                     if np.abs(reward_dict["reward_dist"]) < 0.005 and np.abs(reward_dict["reward_ctrl"]) < 0.05:
    #                         r += 64
    #                         if np.abs(reward_dict["reward_dist"]) < 0.001 and np.abs(reward_dict["reward_ctrl"]) < 0.01:
    #                             r += 128
    #                             if np.abs(reward_dict["reward_dist"]) < 0.0001 and np.abs(reward_dict["reward_ctrl"]) < 0.001:
    #                                 r += 256
    #     r_true += 1
    # else:
    #     r += -0.1


    
    ep_len += 1
    total_main += r


    # Ignore the "done" signal if it comes from hitting the time
    # horizon (that is, when it's an artificial terminal signal
    # that isn't based on the agent's state)
    d = False if ep_len==params["max_ep_len"] else d

    # a = torch.tensor([a], device=device)
    o2 = torch.tensor([o2], device=device)
    r = torch.tensor([r], device=device)
    d = torch.tensor([d], device=device)

    # Store experience to replay buffer
    memory.push(o, a, r, o2, d)
    # Super critical, easy to overlook step: make sure to update 
    # most recent observation!
    o=o2
    ep_ret += r
    
    
    # End of trajectory handling
    if d or (ep_len == params["max_ep_len"]):
        sum_ep_ret = ep_ret
        o, ep_ret, ep_len = env.reset(), 0, 0
        o = torch.tensor([o], device=device)


    # Update handling
    if t >= params["update_after"] and t % params["update_every"] == 0:
        view_bar("steps: ", t, total_steps, 'main of rewards = ', float(ep_ret_main), start_time_rec, 
                r_true, reward_dict["reward_ctrl"], reward_dict["reward_dist"], total_main/t)
        
        nep_log["train/reward"].log(ep_ret_main)
        for i in range(params["update_every"]):

            transitions = memory.sample(params["batch_size"])
            # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
            # detailed explanation). This converts batch-array of Transitions
            # to Transition of batch-arrays.
            batch = Transition(*zip(*transitions))

            # print('batch = ', str(batch))
            update(data=batch)

    # End of epoch handling
    if (t+1) % params["steps_per_epoch"] == 0:
        epoch = (t+1) // params["steps_per_epoch"]
        ep_ret_main = sum_ep_ret/epoch
        # Test the performance of the deterministic version of the agent.
        test_agent(epoch)



o2 =  [ 0.99997906  0.998376   -0.00647075  0.05696805  0.08787639  0.08591351
  2.59731069 -0.0294459   0.12198118 -0.08100713  0.        ]
o2 =  [ 0.99785159  0.99896735  0.06551499  0.04543387  0.08787639  0.08591351
  4.59929561 -1.12185075  0.12123177 -0.06717706  0.        ]
o2 =  [ 0.98977496  0.99991298  0.14263775 -0.01319218  0.08787639  0.08591351
  3.16188234 -4.73033122  0.12017348 -0.05739435  0.        ]
o2 =  [ 0.98422931  0.99685448  0.17689734 -0.07925372  0.08787639  0.08591351
  0.31831648 -1.89356392  0.12001318 -0.05740552  0.        ]
o2 =  [ 0.98067645  0.99441266  0.19563666 -0.10556257  0.08787639  0.08591351
  1.58481199 -0.75250601  0.11973485 -0.05633917  0.        ]
o2 =  [ 0.97671261  0.98814863  0.2145518  -0.15350012  0.08787639  0.08591351
  0.35191682 -4.07141377  0.11958243 -0.05762654  0.        ]
o2 =  [ 0.9772195   0.97578739  0.2122311  -0.21872119  0.08787639  0.08591351
 -0.58633244 -2.57301507  0.11984304 -0.06542135  0.        ]
o2 =  [ 0.982

KeyboardInterrupt: 

In [ ]:
nep_log.stop()

NameError: name 'nep_log' is not defined

In [ ]:
model = ac.q
print("Model_q's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

model = ac.pi
print("Model_pi's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model_q's state_dict:
q.0.weight 	 torch.Size([64, 13])
q.0.bias 	 torch.Size([64])
q.2.weight 	 torch.Size([64, 64])
q.2.bias 	 torch.Size([64])
q.4.weight 	 torch.Size([64, 64])
q.4.bias 	 torch.Size([64])
q.6.weight 	 torch.Size([1, 64])
q.6.bias 	 torch.Size([1])
Model_pi's state_dict:
pi.0.weight 	 torch.Size([64, 11])
pi.0.bias 	 torch.Size([64])
pi.2.weight 	 torch.Size([64, 64])
pi.2.bias 	 torch.Size([64])
pi.4.weight 	 torch.Size([64, 64])
pi.4.bias 	 torch.Size([64])
pi.6.weight 	 torch.Size([2, 64])
pi.6.bias 	 torch.Size([2])


In [ ]:
print("pi_optimizer's state_dict:")
for var_name in pi_optimizer.state_dict():
    print(var_name, "\t", pi_optimizer.state_dict()[var_name])

print("q_optimizer's state_dict:")
for var_name in q_optimizer.state_dict():
    print(var_name, "\t", q_optimizer.state_dict()[var_name])

pi_optimizer's state_dict:
state 	 {0: {'step': 299000, 'square_avg': tensor([[1.3869e-01, 5.7344e-02, 5.4624e-02, 1.1435e-01, 1.3306e-03, 2.6316e-03,
         3.2248e+00, 1.2212e+00, 1.6328e-03, 9.0011e-04, 0.0000e+00],
        [2.2875e-02, 1.2831e-02, 9.8737e-03, 1.9560e-02, 2.9151e-04, 3.2672e-04,
         1.9621e+00, 7.4683e-01, 7.5107e-04, 3.0076e-04, 0.0000e+00],
        [7.4421e-02, 3.0351e-02, 1.3528e-02, 5.1273e-02, 1.2196e-03, 6.9916e-04,
         8.2508e-01, 1.0900e+00, 8.1599e-04, 4.2619e-04, 0.0000e+00],
        [1.5786e-01, 7.6565e-02, 7.4462e-02, 1.5622e-01, 2.2438e-03, 3.0492e-03,
         6.7310e+00, 1.9244e+00, 1.3306e-03, 8.5804e-04, 0.0000e+00],
        [1.6592e-02, 1.8457e-02, 9.9656e-03, 1.0348e-02, 2.0880e-04, 2.0866e-04,
         1.1059e+00, 4.4645e-01, 7.0785e-04, 2.8370e-04, 0.0000e+00],
        [6.4602e-01, 3.3136e-01, 1.2925e-01, 3.4042e-01, 6.4643e-03, 6.8946e-03,
         1.1177e+01, 6.5517e+00, 6.5188e-03, 3.3777e-03, 0.0000e+00],
        [2.1985e-01, 1.2

In [ ]:
now = datetime.now()

current_time = str(now.isoformat())



torch.save({
            'model of ac.q': ac.q.state_dict(),
            'model of ac.pi': ac.pi.state_dict(),
            'q_optimizer_state_dict': q_optimizer.state_dict(),
            'pi_optimizer_state_dict': pi_optimizer.state_dict(),
            
            }, "model_nn/model_nn_%s.pt" % current_time)